## Scrapping coches.net

In [ ]:
import requests
import json
import pandas as pd

La función all_keys_exists comprueba que cuando se accede a la url de un anuncio, ese anuncio contenga todos los campos que se muestran en key_list1 y key_list2. De esta forma si algún anuncio no tiene uno de estos campos no lo incluyo

In [ ]:
def all_keys_exists(car_data1, car_data2):
    key_list1 = ["id", "cubicCapacity", "mainProvince", "drivenWheelsId"]
    key_list2 = ["brand", "model", "km", "power", "year", "fuel_types_id", "transmission",
                 "colors", "body_types", "doors", "seats", "seller_type", "price"]
    
    check1 = all(item in car_data1 for item in key_list1)
    check2 = all(item in car_data2 for item in key_list2)

    if check1 is True and check2 is True:
        return True
    else:
        return False

El scrapping lo hago realizando peticiones directamente a la API de coches.net ya que de esta forma obtengo más información que si solamente realizara el scrapping buscando los campos en la web. La API no devuelve todos los campos que están presentes en un anuncio (cuando se hace click a un resultado) pero si devuelve la dirección URL de los anuncios, por tanto por cada resultado devuelto de la API, accedo a la url del anuncio para obtener el resto de campos

In [ ]:
headers = {
    'authority': 'web.gw.coches.net',
    'accept': 'application/json, text/plain, */*',
    'accept-language': 'es-ES,es;q=0.9,en;q=0.8',
    'content-type': 'application/json',
    'origin': 'https://www.coches.net',
    'referer': 'https://www.coches.net/',
    'sec-ch-ua': '"Google Chrome";v="113", "Chromium";v="113", "Not-A.Brand";v="24"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-site',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36',
    'x-adevinta-amcvid': '57456795200337613064110380547604165577',
    'x-adevinta-channel': 'web-desktop',
    'x-adevinta-euconsent-v2': 'CPpg0xMPr4NVqCBAtAESDECoAP_AAP_AAAiQJbtV_H__bX9v8f7_6ft0eY1f9_r77uQzDhfNs-4F3L_W_LwX_2E7NF36tq4KmR4Eu3LBIUNtHNnUTVmxaokVrzHsak2cpTNKJ-BkkHMRe2dYGF5um5tjeQKZ5_p_d3f52T_9_dv-39z33913v3dd_-_1-Pjde5_9H_v_fRfb-_Ib9_7-_8v8_9_rk2_eX__________-___4JbtV_H__bX9v8f7_6ft0eY1f9_r77uQzDhfNs-4F3L_W_LwX_2E7NF36tq4KmR4Eu3LBIUNtHNnUTVmxaokVrzHsak2cpTNKJ-BkkHMRe2dYGF5um5tjeQKZ5_p_d3f52T_9_dv-39z33913v3dd_-_1-Pjde5_9H_v_fRfb-_Ib9_7-_8v8_9_rk2_eX__________-___4AAA',
    'x-adevinta-session-id': '94e77f33-cada-4b1f-8c39-3fb1f948261b',
    'x-schibsted-tenant': 'coches',
}



#Listado de json con todos los datos de cada coche
final_car_data = []

#Algunas campos son numéricos, en el dataset quiero que se guarden con la variable para poder estudiarlos despues
combustible = {'1': 'diesel', '2': 'gasolina', '3':'electrico', '4': 'hibrido', '5': 'hibrido enchufable'}
driven_wheels = {1: 'total', 3: 'delantera', 4:'trasera'}

#Modificar rango dependiendo de cuantos datos se quieran obtener
for i in range(1,10):
    json_data = {
        'pagination': {
            'page': i,
            'size': 100,
        },
        'sort': {
            'order': 'desc','term': 'relevance',
        },
        'filters': {
            'isFinanced': False,
            'price': {'from': None,'to': None,
            },
            'bodyTypeIds': [],
            'categories': {'category1Ids': [2500,],
            },
            'contractId': 0,
            'drivenWheelsIds': [],
            'environmentalLabels': [],
            'equipments': [],
            'fuelTypeIds': [],
            'hasPhoto': None,
            'hasWarranty': None,
            'hp': {'from': None,'to': None,
            },
            'isCertified': False,
            'km': {'from': None,'to': None,
            },
            'luggageCapacity': {'from': None,'to': None,
            },
            'onlyPeninsula': False,
            'offerTypeIds': [0,2,3,4,5,],
            'provinceIds': [],
            'sellerTypeId': 0,
            'transmissionTypeId': 0,
            'vehicles': [
                {
                    'make': 'TOYOTA','makeId': 46,'model': '','modelId': 0,
                },
            ],
            'year': {'from': None,'to': None,
            },
        },
    }
    response = requests.post('https://ms-mt--api-web.spain.advgo.net/search/listing', headers=headers, json=json_data)
    datos = response.content
    datos = json.loads(datos)
    car_data = datos["items"]
    
    result = []    

    for i, datos in enumerate(car_data):
        #Obtener url del anuncio específico
        url = datos["url"]
        full_url = 'https://www.coches.net' + url

        response = requests.get(full_url, headers=headers)
        codigo_pagina = str(response.content).replace('\\', '')

        #print(full_url)

        if not 'Ups!' in codigo_pagina:
            #Me quedo solo con las cadenas que corresponden al json que me interesa
            sub1 = '"page_category_level1":"coches"'
            idx1 = codigo_pagina.index(sub1)
            sub2 = ',"reset":'
            idx2 = codigo_pagina.index(sub2)

            json_data = codigo_pagina[idx1 + len(sub1) + 1: idx2]

            json_data = '{' + json_data
            final_json = json.loads(json_data)
            
            #Comprobar que exitan todas las keys (hay algún anuncio (pocos) que le faltan algunas, por tanto lo descarto)
            if(all_keys_exists(datos, final_json)):

                #Hay algunos campos que hay que filtrarlos
                transmision = final_json["transmission"].replace('common.transmission_', '')
                color = final_json["colors"].replace('common.color_', '')
                
                row = [datos["id"], final_json["brand"], final_json["model"], final_json["km"], datos["cubicCapacity"], final_json["power"], 
                       final_json["year"], combustible[final_json["fuel_types_id"]], datos["mainProvince"], 
                       transmision, driven_wheels[datos["drivenWheelsId"]], final_json["body_types"], final_json["doors"], color, final_json["seats"], 
                       final_json["seller_type"], final_json["price"], full_url]

                result.append(row)
                
    final_car_data = final_car_data + result

df = pd.DataFrame(final_car_data, columns=['id', 'Marca', 'Modelo', 'Km', 'Cilindrada', 'Caballos', 'Anio', 
                                           'Combustible', 'Provincia', 'Transmision', 'Traccion', 'Tipo', 
                                           'Puertas', 'Color', 'Asientos', 'Tipo de vendedor', 'Precio', 'Url'])
df

In [ ]:
#Guardar dataframe como csv
df.to_csv("car_data.csv", sep=";", index=False)